In [ ]:
'''
The aim is to build a predictive model and find out the sales of each product at a particular store.
.Using this model,BigMart will try to understand the properties of products and stores which play a key role in increasing sales
'''

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,NaN,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [ ]:
'''
adding source col to both test and train
'''
train['source']='train'
test['source']='test'

'''
ignore_index If True, do not use the index values along the concatenation axis. The resulting axis will be labeled 0, …, n - 1. 
This is useful if you are concatenating objects where the concatenation axis does not have meaningful indexing information. 
Note the index values on the other axes are still respected in the join.
'''
data = pd.concat([train, test],ignore_index=True)

In [ ]:
data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380,train
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228,train
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700,train
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800,train
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052,train


In [ ]:
data.describe()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales
count,11765.000000,14204.000000,14204.000000,14204.000000,8523.000000
mean,12.792854,0.065953,141.004977,1997.830681,2181.288914
std,4.652502,0.051459,62.086938,8.371664,1706.499616
min,4.555000,0.000000,31.290000,1985.000000,33.290000
25%,8.710000,0.027036,94.012000,1987.000000,834.247400
50%,12.600000,0.054021,142.247000,1999.000000,1794.331000
75%,16.750000,0.094037,185.855600,2004.000000,3101.296400
max,21.350000,0.328391,266.888400,2009.000000,13086.964800


In [ ]:
data.info()
'''
some values are missing hence total no of entries for all features arent same. Here missing values are to be handled
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                11765 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                10188 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          8523 non-null   float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

'\nsome values are missing hence total no of entries for all features arent same. Here missing values are to be handled\n'

In [ ]:
data.Item_Outlet_Sales = data.Item_Outlet_Sales.fillna(data.Item_Outlet_Sales.mean()) #fillna() - Fill NA/NaN values using the specified method
data.Item_Weight = data.Item_Weight.fillna(data.Item_Weight.mean())
data.Outlet_Size = data.Outlet_Size.fillna('Medium')

In [ ]:
data.info()
'''
now total no of entries for all features are same 
'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Fat_Content           14204 non-null  object 
 3   Item_Visibility            14204 non-null  float64
 4   Item_Type                  14204 non-null  object 
 5   Item_MRP                   14204 non-null  float64
 6   Outlet_Identifier          14204 non-null  object 
 7   Outlet_Establishment_Year  14204 non-null  int64  
 8   Outlet_Size                14204 non-null  object 
 9   Outlet_Location_Type       14204 non-null  object 
 10  Outlet_Type                14204 non-null  object 
 11  Item_Outlet_Sales          14204 non-null  float64
 12  source                     14204 non-null  object 
dtypes: float64(4), int64(1), object(8)
memory usag

'\nnow total no of entries for all features are same \n'

In [ ]:
print(data['Item_Identifier'].value_counts()) # gives the count of each type in this column

map1 = {'FD':'Food', 'NC':'Non-Consumable', 'DR':'Drinks'}

data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0 : 2]).map(map1) # consider only first 2 letters
                                                                                        # feature reductions by combining 2 cols
data['Item_Type_Combined'].value_counts()

'''
The map() method only works on panda series where different types of operation can be applied to the items in the series.
It takes each element in the series and applies the function to it, and returns the transformed series.
Parameter is the method to be used for mapping 
'''

FDS12    10
FDE28    10
FDI56    10
FDE09    10
FDI41    10
         ..
FDM50     7
FDL50     7
FDH58     7
FDR51     7
FDI46     7
Name: Item_Identifier, Length: 1559, dtype: int64


'\nThe map() method only works on panda series where different types of operation can be applied to the items in the series.\nIt takes each element in the series and applies the function to it, and returns the transformed series.\nParameter is the method to be used for mapping \n'

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
'''
we can’t have text in our data if we’re going to run any kind of model on it. So before we can run a model, we need to 
make this data ready for the model. Using le we replace the existing text data with the new encoded data.
'''
le = LabelEncoder()

data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])

ohe = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
'''
Similarly here for other cols
'''
le = LabelEncoder()

for i in ohe:
    data[i] = le.fit_transform(data[i])
    
data

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales,source,Item_Type_Combined,Outlet
0,FDA15,9.30,1,0.016047,Dairy,249.8092,OUT049,1999,1,0,1,3735.138000,train,1,9
1,DRC01,5.92,2,0.019278,Soft Drinks,48.2692,OUT018,2009,1,2,2,443.422800,train,0,3
2,FDN15,17.50,1,0.016760,Meat,141.6180,OUT049,1999,1,0,1,2097.270000,train,1,9
3,FDX07,19.20,2,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,1,2,0,732.380000,train,1,0
4,NCD19,8.93,1,0.000000,Household,53.8614,OUT013,1987,0,2,1,994.705200,train,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14199,FDB58,10.50,2,0.013496,Snack Foods,141.3154,OUT046,1997,2,0,1,2181.288914,test,1,8
14200,FDD47,7.60,2,0.142991,Starchy Foods,169.1448,OUT018,2009,1,2,2,2181.288914,test,1,3
14201,NCO17,10.00,1,0.073529,Health and Hygiene,118.7440,OUT045,2002,1,1,1,2181.288914,test,2,7
14202,FDJ26,15.30,2,0.000000,Canned,214.6218,OUT017,2007,1,1,1,2181.288914,test,1,2


In [ ]:
'''
get_dummies() is used for data manipulation. It converts categorical data into dummy or indicator variables.
after encoding the data now those encodings grouped together to create separate columns
'''
'''
we add dummy values cuz comparing ordinary values is meaningless
'''
data = pd.get_dummies(data, columns = ohe)

In [ ]:
data.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,source,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,Dairy,249.8092,OUT049,1999,3735.1380,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,Soft Drinks,48.2692,OUT018,2009,443.4228,train,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,Meat,141.6180,OUT049,1999,2097.2700,train,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,train,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,Household,53.8614,OUT013,1987,994.7052,train,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            14204 non-null  object 
 1   Item_Weight                14204 non-null  float64
 2   Item_Visibility            14204 non-null  float64
 3   Item_Type                  14204 non-null  object 
 4   Item_MRP                   14204 non-null  float64
 5   Outlet_Identifier          14204 non-null  object 
 6   Outlet_Establishment_Year  14204 non-null  int64  
 7   Item_Outlet_Sales          14204 non-null  float64
 8   source                     14204 non-null  object 
 9   Item_Fat_Content_0         14204 non-null  uint8  
 10  Item_Fat_Content_1         14204 non-null  uint8  
 11  Item_Fat_Content_2         14204 non-null  uint8  
 12  Item_Fat_Content_3         14204 non-null  uint8  
 13  Item_Fat_Content_4         14204 non-null  uin

In [ ]:
data.drop(['Item_Type','Outlet_Establishment_Year'], axis = 1, inplace = True) # no association found useful

train = data.loc[data['source'] == "train"]
test = data.loc[data['source'] == "test"]

test.drop(['Item_Outlet_Sales','source'], axis = 1, inplace = True)
train.drop(['source'], axis = 1, inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
train.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_MRP,Outlet_Identifier,Item_Outlet_Sales,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.016047,249.8092,OUT049,3735.1380,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,0.019278,48.2692,OUT018,443.4228,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.016760,141.6180,OUT049,2097.2700,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.000000,182.0950,OUT010,732.3800,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.000000,53.8614,OUT013,994.7052,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
X_train = train.drop(['Item_Outlet_Sales', 'Outlet_Identifier','Item_Identifier'], axis = 1)
y_train = train.Item_Outlet_Sales

In [ ]:
X_test = test.drop(['Outlet_Identifier','Item_Identifier'], axis=1)

In [ ]:
X_train.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_0,Item_Fat_Content_1,Item_Fat_Content_2,Item_Fat_Content_3,Item_Fat_Content_4,Outlet_Location_Type_0,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_0,Outlet_Size_1,Outlet_Size_2,Item_Type_Combined_0,Item_Type_Combined_1,Item_Type_Combined_2,Outlet_Type_0,Outlet_Type_1,Outlet_Type_2,Outlet_Type_3,Outlet_0,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.016047,249.8092,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
1,5.92,0.019278,48.2692,0,0,1,0,0,0,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
2,17.50,0.016760,141.6180,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,19.20,0.000000,182.0950,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
4,8.93,0.000000,53.8614,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0


In [ ]:
y_train.head()

0    3735.1380
1     443.4228
2    2097.2700
3     732.3800
4     994.7052
Name: Item_Outlet_Sales, dtype: float64

In [ ]:
from sklearn.linear_model import  LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

y_pred

array([1848.53604783, 1472.81670435, 1875.65285894, ..., 1809.18796433,
       3565.6645235 , 1267.46171871])

In [ ]:
#r2_score : It is the amount of the variation in the output dependent attribute which is predictable from the input independent variable(s)

from sklearn.metrics import accuracy_score, r2_score, mean_squared_error
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn import metrics

In [ ]:
r2_score(y_train, regressor.predict(X_train))

0.5635892777270479

In [ ]:
cv_score = cross_val_score(regressor, X_train, y_train, cv = 5, scoring='neg_mean_squared_error')
np.sqrt(np.abs(cv_score))

array([1150.93927648, 1118.68414103, 1112.89657923, 1126.30724065,
       1140.59735737])

In [ ]:
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, regressor.predict(X_train))))

RMSE:  1127.2712261909362


In [ ]:
from sklearn.tree import DecisionTreeRegressor

regressor = DecisionTreeRegressor(max_depth = 20, min_samples_leaf = 300)

regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)
y_pred

array([1673.98398729, 1349.51290433,  471.30684669, ..., 1892.06614452,
       3805.94860417, 1349.51290433])

In [ ]:
tree_accuracy = round(regressor.score(X_train,y_train), 2)

tree_accuracy

0.59

In [ ]:
y_train

0       3735.1380
1        443.4228
2       2097.2700
3        732.3800
4        994.7052
          ...    
8518    2778.3834
8519     549.2850
8520    1193.1136
8521    1845.5976
8522     765.6700
Name: Item_Outlet_Sales, Length: 8523, dtype: float64

In [ ]:
r2_score(y_train, regressor.predict(X_train))

0.5884050821570486

In [ ]:
cv_score = cross_val_score(regressor, X_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
np.sqrt(np.abs(cv_score))

array([1138.77137157, 1109.42501179, 1145.66395939, 1113.2648073 ,
       1129.0816826 ])

In [ ]:
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, regressor.predict(X_train))))

RMSE:  1094.7519246232723


In [ ]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor(n_estimators = 100, max_depth = 10, min_samples_leaf = 50, n_jobs = 4)
regressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=10, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=50,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=4, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [ ]:
y_pred = regressor.predict(X_test)
y_pred

array([1607.31499405, 1372.77762046,  601.43811076, ..., 1796.59611291,
       3663.46956325, 1328.49451935])

In [ ]:
rf_accuracy = round(regressor.score(X_train, y_train), 2)
rf_accuracy

0.63

In [ ]:
r2_score(y_train, regressor.predict(X_train))

0.6270686900103182

In [ ]:
cv_score = cross_val_score(regressor, X_train, y_train, cv = 5, scoring = 'neg_mean_squared_error')
np.sqrt(np.abs(cv_score))

array([1100.60592324, 1079.10692822, 1080.18944294, 1074.06780053,
       1087.78547018])

In [ ]:
print("RMSE: ", np.sqrt(metrics.mean_squared_error(y_train, regressor.predict(X_train))))

RMSE:  1042.065790532782
